In [3]:
import pandas as pd
import numpy as np

In [11]:
def objective_function(num_list:list): 
    return sum(num*num for num in num_list)

In [12]:
nVar = 10
upper_bound = [10]*nVar
lower_bound = [-10]*nVar

In [13]:
fob = objective_function()

TypeError: objective_function() missing 1 required positional argument: 'num_list'